<a id='#start'></a>
### Содержание

<a href='#step1'>1. Подготовка</a>

<a href='#step2'>2. Обучение</a>

<a href='#step3'>3. Выводы</a>

# 1. Подготовка

## 1.1 Загрузка и анализ

In [21]:
import pandas as pd
import numpy as np
import torch
import transformers
from tqdm import notebook
import re

import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

from lightgbm import LGBMClassifier

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\papya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\papya\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\papya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df_tweets = pd.read_csv('/datasets/toxic_comments.csv')
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [3]:
df_tweets.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Оценим баланс классов в нашем датасете.

In [4]:
df_tweets['toxic'].value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Наблюдается дисбаланс классов в сторону нетоксичных комметариев.

## 1.2 Подготовка признаков для обучения моделей

Лемматизацию текста будем проводить с использованием `WordNetLemmatizer` из состава библиотеки NLTK. При этом для корректной лемматизации будем выполнять определение части речи с помощью функции так же из билиотеке NLTK.

In [14]:
wnl = WordNetLemmatizer()

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(text):
    lemm_list = [wnl.lemmatize(w, get_wordnet_pos(w)) for w in text.split()]
    lemm_text = " ".join(lemm_list)
    return lemm_text


def clear_text(text):
    reg = re.sub(r'[^a-zA-Z \']', ' ', text)
    return reg

Поскольку процесс лемматизации занимает продолжительное время, сохраним результаты лемматизации в отдельный файл.

In [52]:
%%time
notebook.tqdm.pandas()
df_tweets['lemm_text'] = df_tweets['text'].progress_apply(lambda x: lemmatize(clear_text(x)))
df_tweets.to_csv('lemm_comments.csv')


Wall time: 2h 19min 22s


Загрузим получившийся на предыдущем шаге файл с лемматизированными сообщениями.

In [2]:
lemm_df = pd.read_csv('lemm_comments.csv', index_col=0)
lemm_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159571 entries, 0 to 159570
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   text       159571 non-null  object
 1   toxic      159571 non-null  int64 
 2   lemm_text  159565 non-null  object
dtypes: int64(1), object(2)
memory usage: 4.9+ MB


In [3]:
lemm_df.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww He match this background colour I'm seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I'm really not try to edit war It's ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can't make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...


Разобьем датафрейм на тренировочную и тестовую выборки, получим из тренировочной корпус и обучим на нем TfidfVectorizer. Затем преобразуем с помощью обученного vectorizer тестовую выборку.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(lemm_df['lemm_text'], lemm_df['toxic'], test_size=0.2)

In [9]:
corpus = X_train.values.astype('U')
stopwords = set(nltk_stopwords.words('english'))

In [11]:
%%time
tfidfv = TfidfVectorizer(stop_words=stopwords)
X_train = tfidfv.fit_transform(corpus)
X_test = tfidfv.transform(X_test.values.astype('U'))

Wall time: 23.3 s


<a href='#start'>К содержанию</a>

# 2. Обучение

## 2.1 Подготовка выборок

Получим три варианта выборок для подбора модели:

- с дисбалансом классов;
- с искусственными данными, подготовленными с помощью SMOTE;
- подвыборка без дисбаланса, полученная с помощью RandomUnderSample.

Несбалансированные выборки были получены на предыдущем этапе работы.

In [13]:
y_train.value_counts()

0    114637
1     13019
Name: toxic, dtype: int64

### 2.1.2 Сбалансированная выборка (SMOTE)

In [14]:
sm = SMOTE(random_state=127)
X_smote, y_smote = sm.fit_sample(X_train, y_train)

In [16]:
y_smote.value_counts()

1    114637
0    114637
Name: toxic, dtype: int64

### 2.1.3 Сбалансированная выборка (RandomUnderSampler)

In [17]:
rus = RandomUnderSampler(random_state=127, sampling_strategy='majority')
X_rus, y_rus = rus.fit_resample(X_train, y_train)
y_rus.value_counts()

1    13019
0    13019
Name: toxic, dtype: int64

## 2.2 Подбор моделей

In [18]:
learn_results = []

### 2.2.1 Baseline

Проведем обучение с кросс-валидацией моделей на 3 вариантах выборок, полученных на предыдущем шаге.

#### 2.2.1.1 Без устранения дисбаланса

In [23]:
%%time

sgd = SGDClassifier(random_state=127)
cv_sgd = cross_val_score(sgd, X_train, y_train, cv=3, scoring='f1')
print(np.mean(cv_sgd))

0.6221827402231385
Wall time: 823 ms


In [19]:
%%time
lgbm = LGBMClassifier(random_state=127, n_jobs=-1)
cv_lgbm = cross_val_score(lgbm, X_train, y_train, cv=3, scoring='f1')
print(np.mean(cv_lgbm))

0.7393032703791081
Wall time: 57 s


In [20]:
%%time
lr = LogisticRegression(random_state=127)
cv_lin_reg = cross_val_score(lr, X_train, y_train, cv=3, scoring='f1')
print(np.mean(cv_lin_reg))

0.7031155733064797
Wall time: 12 s


In [24]:
learn_results.append(['LR', '-', 0.703])
learn_results.append(['LGBM', '-', 0.739])
learn_results.append(['SGD', '-', 0.622])

На данном этапе получили 3 значения F1. SGDClassifier показала крайне низкую метрику, поэтому в дальнейшем будем рассматривать только логистическую регрессию и LGBM.

#### 2.2.1.2 Взвешивание классов

In [25]:
%%time
lr = LogisticRegression(random_state=127, class_weight='balanced')
cv_lin_reg = cross_val_score(lr, X_train, y_train, cv=3, scoring='f1')
print(np.mean(cv_lin_reg))

0.7457721885095698
Wall time: 12.7 s


In [26]:
%%time
lgbm = LGBMClassifier(random_state=127, n_jobs=-1, class_weight='balanced')
cv_lgbm = cross_val_score(lgbm, X_train, y_train, cv=3, scoring='f1')
print(np.mean(cv_lgbm))

0.7305085086695794
Wall time: 57.2 s


In [27]:
learn_results.append(['LR', 'class_weight', 0.746])
learn_results.append(['LGBM', 'class_weight', 0.731])

#### 2.2.1.3 SMOTE

In [28]:
%%time
lr = LogisticRegression(random_state=127)
cv_lin_reg = cross_val_score(lr, X_smote, y_smote, cv=3, scoring='f1')
print(np.mean(cv_lin_reg))

0.9352441582113364
Wall time: 20.2 s


In [29]:
%%time
lgbm = LGBMClassifier(random_state=127, n_jobs=-1)
cv_lgbm = cross_val_score(lgbm, X_smote, y_smote, cv=3, scoring='f1')
print(np.mean(cv_lgbm))

0.9134067726710118
Wall time: 1min 48s


In [30]:
learn_results.append(['LR', 'SMOTE', 0.935])
learn_results.append(['LGBM', 'SMOTE', 0.913])

#### 2.2.1.4 RandomUnderSampler

In [31]:
%%time
lr = LogisticRegression(random_state=127)
cv_lin_reg = cross_val_score(lr, X_rus, y_rus, cv=3, scoring='f1')
print(np.mean(cv_lin_reg))

0.8841738927996318
Wall time: 3.65 s


In [32]:
%%time
lgbm = LGBMClassifier(random_state=127, n_jobs=-1)
cv_lgbm = cross_val_score(lgbm, X_rus, y_rus, cv=3, scoring='f1')
print(np.mean(cv_lgbm))

0.8737494814815708
Wall time: 11.2 s


In [33]:
learn_results.append(['LR', 'RUS', 0.884])
learn_results.append(['LGBM', 'RUS', 0.873])

### Выводы по baseline

In [34]:
models_df = pd.DataFrame(data=learn_results, columns=['Model', 'Balancing', 'F1'])
models_df.sort_values(by='F1', ascending=False)

,Model,Balancing,F1
5,LR,SMOTE,0.935
6,LGBM,SMOTE,0.913
7,LR,RUS,0.884
8,LGBM,RUS,0.873
3,LR,class_weight,0.746
1,LGBM,-,0.739
4,LGBM,class_weight,0.731
0,LR,-,0.703
2,SGD,-,0.622


Проведем подбор гиперпараметров для LGBM с 3 вариантами выборок: с дисбалансов, преобразованной SMOTE и преобразованной RandomUnderSampler.

### 2.2.2 Подбор гиперпараметров

В рамках выполнения работы был осуществлен подбор гиперпараметров для модели LGBMClassifier. Для облегчения проекта расчет удален, однако сохранен результат выводов со значениями F1-меры на кросс-валидации.

#### LGBM без баланса классов

{'num_iterations': 300, 'max_depth': 26, 'learning_rate': 0.19}

F1 = 0.7680098910363958

Wall time: 21min 49s

#### LGBM, обучаемая на выборке, подготовленной с помощью SMOTE

{'num_iterations': 300, 'max_depth': 17, 'learning_rate': 0.28}

F1 = 0.9626676345006341

Wall time: 36min 46s

#### LGBM, обучаемая на выборке, подготовленной с помощью RandomUnderSampler

{'num_iterations': 250, 'max_depth': 29, 'learning_rate': 0.09999999999999999}

F1 = 0.8852458966808792

Wall time: 4min 32

Подбор осуществлялся на следующем наборе параметров:

In [14]:
lgbm_grid = {
    'max_depth': [int(x) for x in range(2, 30, 3)],
    'learning_rate' : [float(x) for x in np.arange(0.01, 0.3, 0.03)],
    'num_iterations' : [100, 150, 200, 250, 300]
}

## 2.3 Тестирование моделей

Для тестирования моделей используем тестовую выборку, имеющую дисбаланс схожий с дисбалансом классов в изначальном датасете.

### 2.3.1 LGBM

In [39]:
%%time
lgbm = LGBMClassifier(random_state=127, num_iterations=300, max_depth=26, learning_rate=0.19)
lgbm.fit(X_train,y_train)
lgbm_pred = lgbm.predict(X_test)
print(f1_score(y_test, lgbm_pred))

0.770600558659218
Wall time: 57.2 s


In [40]:
%%time
lgbm = LGBMClassifier(random_state=127, num_iterations=300, max_depth=17, learning_rate=0.28)
lgbm.fit(X_smote, y_smote)
lgbm_pred = lgbm.predict(X_test)
print(f1_score(y_test, lgbm_pred))

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
0.7683982683982683
Wall time: 1min 14s


In [41]:
%%time
lgbm = LGBMClassifier(random_state=127, num_iterations=250, max_depth=29, learning_rate=0.099)
lgbm.fit(X_rus, y_rus)
lgbm_pred = lgbm.predict(X_test)
print(f1_score(y_test, lgbm_pred))

[LightGBM] [Warning] Accuracy may be bad since you didn't set num_leaves and 2^max_depth > num_leaves
0.7096106583883068
Wall time: 11 s


### 2.3.2 Логистическая регрессия

In [44]:
%%time
lr = LogisticRegression(random_state=127)
lr.fit(X_smote, y_smote)
lr_pred = lr.predict(X_test)
print(f1_score(y_test, lr_pred))

0.67844168858825
Wall time: 8.41 s


In [43]:
%%time
lr = LogisticRegression(random_state=127, class_weight='balanced')
lr.fit(X_train,y_train)
lr_pred = lr.predict(X_test)
print(f1_score(y_test, lr_pred))

0.743418360387396
Wall time: 7.11 s


In [45]:
%%time
lr = LogisticRegression(random_state=127)
lr.fit(X_rus, y_rus)
lr_pred = lr.predict(X_test)
print(f1_score(y_test, lr_pred))

0.6946122860020141
Wall time: 1.56 s


<a href='#start'>К содержанию</a>

# 3. Выводы

На этапе тестирования были проверены 6 вариантов моделей - 3 для **логистической регрессии** и 3 для **LGBM**.

Все варианты моделей, использовавших для обучения выборки без дисбаланса, показали ухудшение значения F1-меры на тестовой выборке в сравнении с кросс-валидацией.
 Близкой к целевой метрике, но все еще недостаточной, оказалась модель логистической регрессии с параметром взвешивания классов при обучении. Подобное значительное ухудшение метрик может обозначать переобучение на данных, сильно отличающихся от тестовых с точки зрения баланса.

Целевое значение *F1=0.75* преодолели две модели **LGBM**, лучшей же стала модель, обученная на несбалансированной выборке, значение метрики F1 на тестовой выборке для нее оказалось равным **0.77**.

<a href='#start'>К содержанию</a>